In [1]:
#CNN_GTzan-2D-75w-LoadAndTest-110250

import numpy as np
import soundfile as sf
from numpy import *
from keras import regularizers
import os, sys
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten
from keras.callbacks import TensorBoard
from keras.utils import np_utils, to_categorical
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
import keras.initializers as init

os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
config = tf.ConfigProto( )
config.gpu_options.allow_growth = True
sess   = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)

Using TensorFlow backend.


In [2]:
# Controling Hyperparameters
nb_classes = 10
frame_size = 110250

In [3]:
#Indicate folds
train_fold = [2, 3]
test_fold = 1

#Indicate mf or stft
mf_or_stft = "stft"
#If stft, we need a "_stft" at the end, otherwise empty ""...
none_or_stft = "_stft"
#Attacked: "test_" or "BIM_" or "FGSM_"
attacked = "BIM_"


In [4]:
X_valid = np.load(str("folds_"+mf_or_stft+"/2_GTzan_Xs_"+attacked+"fold"
                      +str(test_fold)+"_110250_75_frozen"+none_or_stft+".npy"))

In [5]:
Y_valid = np.load( str("folds_"+mf_or_stft+"/2_GTzan_Ys_test_fold"
                       +str(test_fold)+"_110250_75_frozen"+none_or_stft+".npy"))
t_valid = np.load( str("folds_"+mf_or_stft+"/2_GTzan_ts_test_fold"
                       +str(test_fold)+"_110250_75_frozen"+none_or_stft+".npy"))
s_valid = np.load( str("folds_"+mf_or_stft+"/2_GTzan_ss_test_fold"
                       +str(test_fold)+"_110250_75_frozen"+none_or_stft+".npy"))

In [6]:
X_valid.shape, Y_valid.shape, t_valid.shape, s_valid.shape

((7138, 257, 431), (7138, 10), (7138,), (7138,))

In [7]:
f = X_valid.shape[1]
g = X_valid.shape[2]

# Adapt 1D data to 2D CNN
X_valid = np.squeeze( X_valid )
X_valid = np.expand_dims( X_valid, axis = 3 )

print(X_valid.shape, Y_valid.shape)

(7138, 257, 431, 1) (7138, 10)


In [8]:
def model_generator_GTzannet2D_1a():
    
    from keras.layers      import Input, Dense, Conv2D, AveragePooling1D, LeakyReLU, MaxPool2D, Flatten
    from keras.layers.core import Dropout
    from keras.models      import Model
    from keras             import initializers, optimizers, regularizers
    from keras.callbacks   import ModelCheckpoint
    from keras.utils       import multi_gpu_model
    
    from keras.layers.normalization import BatchNormalization
        
    import keras.initializers as init
    
    from kapre.utils          import Normalization2D
    from kapre.augmentation   import AdditiveNoise    
    
    sr = 22050
    
    inp   = Input(shape = (f, g, 1)) 
    #----------------------
    conv1  = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(inp)
    norm1  = BatchNormalization()(conv1)
    #----------------------
    conv2  = Conv2D(filters = 32, kernel_size = (3, 3) )(norm1)
    act2   = LeakyReLU(alpha = 0.2)(conv2)
    pool2  = MaxPool2D(pool_size = 2, strides = 2)(act2)
    drop2  = Dropout(0.05)(pool2)
    #----------------------
    conv3  = Conv2D(filters = 64, kernel_size = (3, 3) )(drop2)
    act3   = LeakyReLU(alpha = 0.2)(conv3)
    #----------------------
    conv4  = Conv2D(filters = 64, kernel_size = (3, 3) )(act3)
    act4   = LeakyReLU(alpha = 0.2)(conv4)
    pool4  = MaxPool2D(pool_size = 4, strides = 2)(act4)
    #----------------------
    flat   = Flatten()(pool4)
    #----------------------      
    dense3 = Dense(512, activation='relu', kernel_initializer = initializers.glorot_uniform(seed = 0))(flat)
    #dense3 = Dense(1024, activation='relu', kernel_initializer = initializers.glorot_uniform(seed = 0))(flat)
    drop3  = Dropout(0.80)(dense3)    
    #----------------------
    dense4 = Dense(nb_classes, activation='softmax')(drop3)
    #----------------------

    model  = Model(inp, dense4)

    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = 1e-08, decay = 0.0),
                  metrics = ['accuracy'] )
    
    model.summary()
    
    return model

In [9]:
# Re-generating the model 
model = model_generator_GTzannet2D_1a()
path = str( "weights/weights_3_GTzan_3f_fold"+str(train_fold[0])
           +"-"+str(train_fold[1])+"_20p_110250_75_frozen"+none_or_stft+".best.hdf5" )
model.load_weights(path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 257, 431, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 255, 429, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 255, 429, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 427, 32)      9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 253, 427, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 213, 32)      0         
___________________________________________

In [ ]:
# Evaluating the model
print(path)
print("Testing on test set of fold :" + str(test_fold)) 
print(model.evaluate(X_valid, Y_valid, verbose = 1))
print(model.metrics_names)

weights/weights_3_GTzan_3f_fold2-3_20p_110250_75_frozen_stft.best.hdf5
Testing on test set of fold :1

 160/7138 [..............................] - ETA: 42:17

In [ ]:
X_valid.shape, Y_valid.shape

In [ ]:
t_valid.min(), t_valid.max(), s_valid.min(), s_valid.max()

In [ ]:
len(t_valid), len(s_valid) 

In [ ]:
# Zips t_valid, s_valid with value of their original index 

indexes = list(range(0,len(t_valid)))
sorted_zip = sorted(zip(t_valid, s_valid, indexes))
sorted_t_valid, sorted_s_valid, sorted_indexes = zip(*sorted_zip)


In [ ]:
#Combining similar segments

big_matrix = [] # Stores list of windows for each track, with the track label

print("Testing on test set of fold number: " + str(test_fold))
print_batch = 20       # Will print a message every 20 loops

h = int(s_valid.max()) # all frames available
beg = t_valid.min()    # smallest id value out of the track values
end = t_valid.max()    # highest id value out of the track values
similar_items_res_sum = list()
y_target = None

for i in range(0, len(sorted_t_valid)):
    
    if (beg != sorted_t_valid[i]):
        
        if (beg%print_batch == 0):
            print("Done with:", beg, "out of", end)
        
        beg = sorted_t_valid[i]
        big_matrix.append([similar_items_res_sum, y_target])
        similar_items_res_sum = list()
    
    j = sorted_indexes[i] # Index of the original unsorted list
    
    y_proba   = model.predict(X_valid[j:j+1])
    #y_predict = y_proba.argmax()
    y_target  = Y_valid[j:j+1].argmax(axis=-1)
    
    similar_items_res_sum.append(y_proba)
    
big_matrix.append([similar_items_res_sum, y_target]) #Append last list

len_big_matrix = len(big_matrix)

print("Done testing - Total samples:", len_big_matrix)

In [ ]:

#Insert the number of splits (Example: 5 splits for 21 frames -> 1, 6, 12, 18, 21)

splits = [1, 6, 12, 18, 21]
        
percentage_vot = []
percentage_sum  = []

for i in range (len(splits)):
    percentage_vot.append(0)
    percentage_sum.append(0)
i = 0
    
for obj_similiar_items_res_sum in big_matrix:

    similar_items_res_sum = obj_similiar_items_res_sum[0]
    y_target = obj_similiar_items_res_sum[1]
    
    beg = 0
    index = 0
    lens = len(similar_items_res_sum) #To access the list of similar objects
    candidates_vot = np.zeros(nb_classes)
    candidates_sum = np.ones(nb_classes)
    
    for split in splits:
        
        current_split = split
        end = min(lens, split)
            
        for n in range(beg, end):
            candidates_vot[similar_items_res_sum[n].argmax()] += 1 #voting
            candidates_sum = candidates_sum + similar_items_res_sum[n] #sum
            
        if (np.argmax(candidates_vot) == y_target):
            percentage_vot[index] += 1
        if (np.argmax(candidates_sum) == y_target):
            percentage_sum[index] += 1
            
        candidates_sum = candidates_sum.copy()
        candidates_vot = candidates_vot.copy()
            
        beg = current_split
        index += 1
        
    i += 1
    if (i%print_batch == 0):
        print("Done with:", i, "out of", len_big_matrix)
    

In [ ]:
###### VOTING RULE
i = 0
for split in percentage_vot:
    samples = len(big_matrix)
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1

In [ ]:
# SUM RULE
i = 0
for split in percentage_sum:
    samples = len(big_matrix)
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1